In [ ]:
##Longhand - M. Cook - 2022
##Takes text corpus and returns immersive visualization
##https://github.com/Cook4986/Longhand

In [ ]:
#Takes Bag-of-Words and returns json dump of common nouns, Sketchfab models/uids, and relative percentage of occurance
import spacy
from collections import Counter
import json
import requests
from requests.exceptions import RequestException
import time

#select language models here: https://spacy.io/models
#nlp = spacy.load("it_core_news_lg")
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")
nlp.max_length = 10000000
stopwords = nlp.Defaults.stop_words

#I/O
BoW = "...txt"
output = ".../Longhand/objects.txt"

#declarations
model_size = 100000 #face count
SKETCHFAB_API_URL = "https://api.sketchfab.com/v3/search?type=models&count=1&max_face_count=" + str(model_size) #note count parameter
API_TOKEN = '...' #keep private
results = 20 #target number of models, potentially limited by NER outputs
start = time.time()

#data structures
nouns = [] #nouns in Bag-of-Words
entities = [] #named entities
freqs = [] # noun appearance frequencies
objects = {} # key = common nouns; value(s) = [relative percentage of total objects, UID, model name, URL]

#parse Bag-of-Words parts-of-speech with SpaCy
with open(BoW, encoding="utf-8") as file:
    print("Tokenizing text...")
    iliad = file.read()
document = nlp(iliad)
document.text

#collate nouns in corpus
for token in document:
    token = token.lemma_
    if (token not in stopwords) & (token.pos_ == 'NOUN'): 
            nouns.append(token.text)

#collate named entities, limited to categories with presumed sketchfab correspondences
labelsEN = ["PRODUCT"] #["PRODUCT""EVENT","FAC","WORK_OF_ART","LOC","NORP","GPE","ORG"] #english model, add "PERSON"
labelsIT = ["LOC"]# "MISC", "ORG","PER"] #italian model, add "PER"
for entity in document.ents:
    for label in labelsIT:
        if (token not in stopwords) & (entity.label_ == label):
            entities.append(entity.text)

#frequency & dictionary creation with common words
word_freq = Counter(nouns) #nouns or entities
common = word_freq.most_common(results)
for word in common:
    freqs.append(word[1])
Sum = sum(freqs)
for word in common:
    flowt = (word[1] / Sum) * 100
    percentage = round(flowt, 2)
    objects[word[0]] = [percentage]
print("nlp step complete")
print("\n")
print("dictionary created")
print("\n")

#Sketchfab API payload function 
##From https://sketchfab.com/developers/data-api/v3/python#example-python-model
def _get_request_payload(*, data=None, files=None, json_payload=False):
    """Helper method that returns the authentication token and proper content type depending on
    whether or not we use JSON payload."""
    data = data or {}
    files = files or {}
    headers = {'Authorization': 'Token {}'.format(API_TOKEN)}
    if json_payload:
        headers.update({'Content-Type': 'application/json'})
        data = json.dumps(data)
    return {'data': data, 'files': files, 'headers': headers}

#query sketchfabs with nouns in BoW and return/write list of uids + model names
for key in objects.keys():
    key = str(key)
    print("Searching: " + key)
    print("\n")
    query = ("&q="+(key)+"&downloadable=true")
    search_endpoint = f'{SKETCHFAB_API_URL + query}'
    payload = _get_request_payload() 
    response = requests.get(search_endpoint, **payload)
    data = response.json()
    print("\n")
    for item in range(len(data['results'])):
        url = (data['results'][item]['uri'])
        uid = (data['results'][item]['uid'])
        name = (str((data['results'][item]['name'])))
        size = int(data['results'][item]['faceCount'])
        if key.lower() in name.lower():
            objects[key] += [name, uid, url]
            print("the following model has been located: ")
            print(str((data['results'][item]['name']))+(" \nuid: ")+(data['results'][item]['uid']))
            if size > 50000:
                print("\n")
                print("Warning: Model size exceeds 50000 faces.")
            print("\n")
            
#write dictionary to disk 
with open(output, 'w') as file:
    file.write(json.dumps(objects)) 
file.close()
print(str(results) + " most common objects in corpus written to disk")

#print number of suitable matches
hits = []
for key,value in objects.items():
    if len(value) > 2:
        hits.append(key)
print((str(len(hits))) + " suitable models located on Sketchfab")
print("\n")

#terminate program
end = time.time()
print(str(end - start) + " seconds elapsed" )
print("\n")
print("have a nice day")


In [ ]:
##Launches Blender from terminal and initiates model download script
!/Applications/Blender.app/Contents/MacOS/Blender --python .../Longhand/Longhand_downloader.py


In [ ]:
#Notes
##Tested on 9.2 MB text file (cookbooks) w/ en_core_web_lg model. Mac Desktop. Timed out.
##Add "working directory", across scripts, to automate declarations